# Rotating convection in a cylinder

In [ ]:
from cylinderConvection.cylinderConvection import cylConvection
import numpy as np

# Examples

## 1. Quick example

Create a class instance each time you set the tank aspect ratio Γ (Γ = diameter/height), the number of azimuthal velocity wavenumbers M, radial wavenumbers K, and passive temperature wavenumbers J.  

For this example, it's sufficient to limit our search to the first 20 x 20 velocity modes and include only the first 20 temperature modes.

In [ ]:
RotatingCylinder = cylConvection(Γ = 4, M = 20, K = 20, J = 20)

Compute the minimum Rayleigh numbers (Ra = α ΔT g H^3/(ν κ)) for oscillatory and wall convection, given Ekman (E = ν/(2 Ω H^2)) and Prandtl (Pr = ν/κ) numbers. Print values.

In [ ]:
RotatingCylinder.minimizeRayleigh(E = 1e-4, Pr = 0.025, printVals=True)

## 2. Looping over a range of Ekman numbers

Create a class instance each time you set Γ, M, K, J (should take ~50 seconds):

In [ ]:
RotatingCylinder = cylConvection(Γ = 2, M = 90, K = 90, J = 90)

Compute the minimum Rayleigh number for each Ekman number in an array of values (should take ~2 seconds):

In [ ]:
Pr = 0.025
numE = 100;
EArr = np.logspace(-8,-3,num=numE)
RaOCylArr = np.zeros(len(EArr))
RaWArr = np.zeros(len(EArr))
m0cArr = np.zeros(len(EArr))
k0cArr = np.zeros(len(EArr))

for i in range(len(EArr)):
    E = EArr[i]
    Rac, R1c, σ1c, m0c, k0c, RaWall, RWall, σWall, mWall = RotatingCylinder.minimizeRayleigh(E = E,Pr = Pr)
    RaOCylArr[i] = Rac
    RaWArr[i] = RaWall
    m0cArr[i] = m0c
    k0cArr[i] = k0c

Plot:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

colors = ['#4E8FC8','#D9A900','#C50230']

E_filt = EArr[0:-21]
k0c_filt = k0cArr[0:-21]
m,b = np.polyfit(np.log(E_filt),np.log(k0c_filt),1)

f, (ax1, ax2) = plt.subplots(2, 1,figsize=(7,7))
title = f'$\\Gamma = {RotatingCylinder.Γ:.2f},\\quad Pr = {Pr:.3f},'
title += f'\\quad M={RotatingCylinder.M:.0f}, \\quad K={RotatingCylinder.K:.0f}, \\quad J={RotatingCylinder.J:.0f}$'
ax1.set_title(title)
ax1.loglog(EArr,RaWArr,'k--',label='$Ra_W$')
ax1.loglog(EArr,RaOCylArr,color=colors[0],linestyle='--',label='$Ra_O^{cyl}$')
ax1.invert_xaxis()
ax1.set_xlim(EArr[-1],EArr[0])
ax1.set_ylim(1e4,1e11)
ax1.grid()
ax1.set_xlabel('$E$')
ax1.set_ylabel('$Ra$')
ax1.legend()

ax2.loglog(EArr,m0cArr,label='$m_{0c}$',color='#915eb5')
ax2.loglog(EArr,k0cArr,label='$k_{0c}$',color='#e39600')
ax2.loglog(EArr[EArr<=E_filt[-1]],np.exp(np.log(EArr[EArr<=E_filt[-1]])*m + b),color='gray',linestyle='--',label=f'$k$ = {np.exp(b):.2f} $E^{{{m:.2f}}}$')
ax2.invert_xaxis()
ax2.set_yticks(np.array([1,5,10,50,100]),labels=['1','5','10','50','100'])
ax2.grid(color='#D3D3D3')
ax2.set_xlim(EArr[-1],EArr[0])
ax2.set_xlabel('$E$')
ax2.set_ylabel('critical wavenumber (oscillatory)')
ax2.legend(loc='upper left')
fname = f'Γ={RotatingCylinder.Γ:.2f}-Pr={Pr:.3f}-M={RotatingCylinder.M:.0f}-K={RotatingCylinder.K:.0f}-J={RotatingCylinder.J:.0f}.jpg'
#plt.savefig(fname,dpi=300)
plt.show()

# Check against values from [Zhang & Liao (2017)](https://doi.org/10.1017/9781139024853)

In [ ]:
EZL,PrZL,ΓZL,k0cZL,m0cZL,R1cZL,σ1cZL = np.loadtxt('Zhang&Liao2017Values.csv',skiprows=1,delimiter=',',unpack=True)
ΓZL = np.unique(ΓZL).item()

In [ ]:
RotatingCylinder = cylConvection(Γ = ΓZL, M = 25, K = 25, J = 25)

In [ ]:
R1cArr = np.zeros(len(EZL))
σ1cArr = np.zeros(len(EZL))
m0cArr = np.zeros(len(EZL))
k0cArr = np.zeros(len(EZL))

for i in range(len(EZL)):
    E = EZL[i]
    Pr = PrZL[i]
    Rac, R1c, σ1c, m0c, k0c, RaWall, RWall, σWall, mWall = RotatingCylinder.minimizeRayleigh(E = E,Pr = Pr)
    R1cArr[i] = R1c
    σ1cArr[i] = σ1c
    m0cArr[i] = m0c
    k0cArr[i] = k0c

numWrongR1c = np.sum(np.abs((R1cZL - R1cArr)/R1cZL) > 1e-3)
numWrongσ1c = np.sum(np.abs((σ1cZL - σ1cArr)/σ1cZL) > 1e-3)
numWrongm0c = np.sum(np.abs(m0cArr-m0cZL))
numWrongk0c = np.sum(np.abs(k0cArr-k0cZL))

if (numWrongR1c+numWrongσ1c+numWrongm0c+numWrongk0c) == 0:
    msg = f'Computed values are within 0.1% of published values (at'
    msg+= f' M = {RotatingCylinder.M:.0f}, K = {RotatingCylinder.K:.0f}, and J = {RotatingCylinder.J:.0f})'
    print(msg)
else:
    print("Something's wrong.")